# GSP Final Project
In this notebook, we will demonstrate estimation of graph signals using a GSP-based technique vs.  a deep-learning-based technique. 

In [5]:
import util
import torch
import torch.nn as nn
import numpy as np
import scipy.sparse as sp
import scipy.io as sio
import torch_geometric as tg
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Apr 21 12:50:26 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   40C    P8              2W /   55W |     682MiB /   8188MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Task

$$\mathbf{x} \longrightarrow  \boxed{\text{Physical Model}} \longrightarrow \mathbf{y}  \longrightarrow \boxed{\text{Estimator}}  \longrightarrow \hat{\mathbf{x}}$$ 

### Physical Model
$$ \mathbf{y} = \mathbf{g}(\mathbf{x};\mathbf{L}) + \mathbf{w} $$

- $\mathbf{x} \sim p(\mathbf{x})$.
- $\mathbf{g}$ - a non-linear measurement function.
- $\mathbf{L}$ - the Laplacian matrix of the graph.
 
### Estimator
- $\mathbf{y}$ - input of the estimator
- $\mathbf{x}$ - ground-truth corresponding label
- The estimator is given a dataset pairs of $\{\mathbf{x_t}, \mathbf{y_t} \}_t$ for training.

**Goal**: The estimator should recover $\mathbf{x}$ out of $\mathbf{y}$ with minimum MSE.
#### GSP-based

Use a GSP-based technique for estimation.

#### GNN-based

Use a GNN deep-learning architecture for estimation.

**NOTE:**  $\mathbf{y}$ is the input to the DP model and $\mathbf{x}$ is the output (i.e., the label) not vise versa!

## Part 1: Load data

In [3]:
filename = 'grid_data_ieee118cdf.mat'
data = sio.loadmat(filename)
Y = data['Y']
N = Y.shape[0]

In [7]:
class GNN(torch.nn.Module):
    def __init__(self, num_features, K = 10):  # K is the order of the Chebyshev polynomial
        super(GNN, self).__init__()
        self.conv1 = tg.nn.ChebConv(num_features, 118, K=K, normalization=None)
        self.relu1 = nn.ReLU()

    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight)
        x = self.relu1(x)
        return x


## Define GNN training function

In order to run on GPU via CUDA set device accordingly.

In [ ]:
def train_model(model, train_data, valid_data, batch_size=20, valid_batch_size = 50, epochs=40, lr=0.001, weight_decay=1e-4, path=None, device='cpu'):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    train_mse, valid_mse = [], []
    
    for epoch in range(epochs):
        idx = np.random.permutation(len(train_data))
        train_data = train_data[idx]
        
        # Training
        train_loss = 0
        iters = 0
        for i in range(0, len(train_data), batch_size):
            model.eval()
            
            yt = train_data.y[i: i + batch_size]
            yt = torch.tensor(yt, dtype=torch.float32, device=device)
            
            xt = train_data.x[i: i + batch_size]
            xt = torch.tensor(xt, dtype=torch.float32, device=device)

            optimizer.zero_grad()
            xt_hat = model(yt)
            loss = criterion(xt,xt_hat)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            iters += 1
        train_mse.append(train_loss / iters)
        
        # Validation
        model.eval()
        valid_loss = 0
        iters = 0
        with torch.no_grad():
            for i in range(0, len(valid_data), valid_batch_size):
                yt = train_data.y[i: i + batch_size]
                yt = torch.tensor(yt, dtype=torch.float32, device=device)
            
                xt = train_data.x[i: i + batch_size]
                xt = torch.tensor(xt, dtype=torch.float32, device=device)
                
                xt_hat = model(yt)
                loss = criterion(xt,xt_hat)
                valid_mse += loss.item()
                iters += 1
            valid_mse.append(valid_mse / iters)
        
        if valid_mse[-1] == max(valid_mse) and path is not None:
            print("Current State Saved")
            torch.save(model.state_dict(), path)
        
        print(f"Epoch: {epoch}, Train MSE {train_mse[-1]}, Validation MSE {valid_mse[-1]}")
        
def plot_learning_curve(train_mse, valid_mse):
    plt.figure(figsize=(10, 5))
    plt.plot(train_mse, label='Training MSE', color='blue')
    plt.plot(valid_mse, label='Validation MSE', color='black')
    plt.title('Training and Validation Losses')
    plt.xlabel('Epochs')
    plt.ylabel('MSE')
    plt.legend()
    plt.show()